In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## படி 1: கட்டமைக்கப்பட்ட வெளியீடுகளுக்கான Pydantic மாதிரிகளை வரையறுக்கவும்

இந்த மாதிரிகள் ஏஜென்ட்கள் திருப்பி அளிக்கும் **schema**-ஐ வரையறுக்கின்றன. Pydantic உடன் `response_format`-ஐ பயன்படுத்துவதால் கிடைக்கும் நன்மைகள்:
- ✅ வகை-பாதுகாப்பான தரவுகளை எளிதாக எடுக்க முடியும்
- ✅ தானியங்கச் சரிபார்ப்பு
- ✅ இலவச உரை பதில்களில் பிழைகள் இல்லாமல்
- ✅ புலங்களின் அடிப்படையில் எளிதான நிபந்தனை வழிமுறைகள்


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## படி 2: ஹோட்டல் முன்பதிவு கருவியை உருவாக்கவும்

இந்த கருவி **availability_agent** மூலம் அறைகள் கிடைக்கிறதா என்பதை சரிபார்க்க அழைக்கப்படும். `@ai_function` அலங்காரத்தை பயன்படுத்தி:
- Python செயல்பாட்டை AI அழைக்கக்கூடிய கருவியாக மாற்றவும்
- LLM க்கான JSON schema ஐ தானாக உருவாக்கவும்
- அளவுரு சரிபார்ப்பை கையாளவும்
- முகவர்களால் தானாக அழைக்கப்படுவதற்கு அனுமதிக்கவும்

இந்த டெமோவிற்காக:
- **ஸ்டாக்ஹோம், சீட்டில், டோக்கியோ, லண்டன், ஆம்ஸ்டர்டாம்** → அறைகள் உள்ளன ✅
- **மற்ற அனைத்து நகரங்கள்** → அறைகள் இல்லை ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## படி 3: வழிமாற்றத்திற்கான நிலைமைகள் செயல்பாடுகளை வரையறுக்கவும்

இந்த செயல்பாடுகள் முகவரின் பதிலை ஆய்வு செய்து, வேலைப்பாடில் எந்த பாதையை எடுத்துச் செல்ல வேண்டும் என்பதை தீர்மானிக்கின்றன.

**முக்கிய முறை:**
1. செய்தி `AgentExecutorResponse` ஆக உள்ளதா என்பதை சரிபார்க்கவும்
2. அமைந்த வெளியீட்டை (Pydantic மாதிரி) பகுப்பாய்வு செய்யவும்
3. வழிமாற்றத்தை கட்டுப்படுத்த `True` அல்லது `False` ஐ திருப்பவும்

வேலைப்பாடு **எட்ஜ்களில்** இந்த நிலைமைகளை மதிப்பீடு செய்து, அடுத்ததாக எந்த செயல்பாட்டாளரை அழைக்க வேண்டும் என்பதை தீர்மானிக்கும்.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## படி 4: தனிப்பயன் காட்சி செயல்படுத்தியை உருவாக்கவும்

செயல்படுத்திகள் என்பது மாற்றங்கள் அல்லது பக்க விளைவுகளைச் செய்யும் வேலைப்பாடுகளின் கூறுகள். இறுதி முடிவை காட்டும் தனிப்பயன் செயல்படுத்தியை உருவாக்க `@executor` அலங்காரத்தை பயன்படுத்துகிறோம்.

**முக்கிய கருத்துக்கள்:**
- `@executor(id="...")` - ஒரு செயல்பாட்டை வேலைப்பாடுச் செயல்படுத்தியாக பதிவு செய்கிறது
- `WorkflowContext[Never, str]` - உள்ளீடு/வெளியீட்டுக்கான வகை குறிப்புகள்
- `ctx.yield_output(...)` - இறுதி வேலைப்பாடின் முடிவை வழங்குகிறது


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## படி 5: சூழல் மாறிகள் ஏற்றவும்

LLM கிளையண்டை அமைக்கவும். இந்த எடுத்துக்காட்டு கீழ்வரும்வற்றுடன் வேலை செய்கிறது:
- **GitHub மாடல்கள்** (GitHub டோக்கனுடன் இலவச நிலை)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## படி 6: அமைப்பான வெளியீடுகளுடன் AI முகவர்களை உருவாக்குதல்

நாம் **மூன்று சிறப்பு முகவர்களை** உருவாக்குகிறோம், ஒவ்வொன்றும் `AgentExecutor` மூலம் மூடப்பட்டுள்ளன:

1. **availability_agent** - ஹோட்டல் கிடைக்குமா என்பதை கருவியைப் பயன்படுத்தி சரிபார்க்கிறது
2. **alternative_agent** - மாற்று நகரங்களை பரிந்துரைக்கிறது (அறைகள் இல்லாதபோது)
3. **booking_agent** - முன்பதிவை ஊக்குவிக்கிறது (அறைகள் கிடைக்கும் போது)

**முக்கிய அம்சங்கள்:**
- `tools=[hotel_booking]` - முகவருக்கு கருவியை வழங்குகிறது
- `response_format=PydanticModel` - அமைப்பான JSON வெளியீட்டை கட்டாயமாக்குகிறது
- `AgentExecutor(..., id="...")` - வேலைப்பாடுகளில் பயன்படுத்த முகவரை மூடுகிறது


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## படி 7: நிபந்தனை விளிம்புகளுடன் பணிச்சூழலை உருவாக்கவும்

இப்போது நிபந்தனை வழிமுறைகளுடன் வரைபடத்தை உருவாக்க `WorkflowBuilder` ஐ பயன்படுத்துகிறோம்:

**பணிச்சூழல் அமைப்பு:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**முக்கிய முறைகள்:**
- `.set_start_executor(...)` - தொடக்க புள்ளியை அமைக்கிறது
- `.add_edge(from, to, condition=...)` - நிபந்தனை விளிம்பை சேர்க்கிறது
- `.build()` - பணிச்சூழலை இறுதியாக முடிக்கிறது


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## படி 8: சோதனை வழக்கு 1 - கிடைக்காத நகரம் (பாரிஸ்)

கிடைக்காத பாதையை சோதிக்க பாரிஸில் ஹோட்டல்களை கோருவோம் (எங்கள் சிமுலேஷனில் எந்த அறைகளும் இல்லை).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## படி 9: சோதனை வழக்கு 2 - கிடைக்குமிடம் உள்ள நகரம் (ஸ்டாக்ஹோம்)

இப்போது ஸ்டாக்ஹோம் நகரத்தில் (எங்கள் சிமுலேஷனில் அறைகள் உள்ள நகரம்) ஹோட்டல்களை கோருவதன் மூலம் **கிடைக்குமிடம்** பாதையை சோதிப்போம்.


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## முக்கிய குறிப்புகள் மற்றும் அடுத்த படிகள்

### ✅ நீங்கள் கற்றுக்கொண்டவை:

1. **WorkflowBuilder முறை**
   - `.set_start_executor()` பயன்படுத்தி தொடக்க புள்ளியை வரையறுக்கவும்
   - `.add_edge(from, to, condition=...)` மூலம் நிபந்தனை வழிமுறையை அமைக்கவும்
   - `.build()` அழைத்து வேலைப்பாட்டை இறுதிப்படுத்தவும்

2. **நிபந்தனை வழிமுறை**
   - நிபந்தனை செயல்பாடுகள் `AgentExecutorResponse` ஐ ஆய்வு செய்கின்றன
   - அமைப்பான வெளியீடுகளை பகுத்து வழிமுறைகளை தீர்மானிக்கவும்
   - ஒரு வழியை செயல்படுத்த `True` ஐ திருப்பவும், தவிர்க்க `False` ஐ திருப்பவும்

3. **கருவி ஒருங்கிணைப்பு**
   - Python செயல்பாடுகளை AI கருவிகளாக மாற்ற `@ai_function` பயன்படுத்தவும்
   - ஏஜென்ட்கள் தேவையான போது கருவிகளை தானாக அழைக்கின்றன
   - கருவிகள் JSON ஐ திருப்புகின்றன, அதை ஏஜென்ட்கள் பகுத்தறிய முடியும்

4. **அமைப்பான வெளியீடுகள்**
   - தரவுகளை பாதுகாப்பாக எடுக்க Pydantic மாதிரிகளை பயன்படுத்தவும்
   - ஏஜென்ட்களை உருவாக்கும்போது `response_format=MyModel` அமைக்கவும்
   - பதில்களை `Model.model_validate_json()` மூலம் பகுத்தறியவும்

5. **தனிப்பயன் செயல்படுத்திகள்**
   - `@executor(id="...")` பயன்படுத்தி வேலைப்பாட்டு கூறுகளை உருவாக்கவும்
   - செயல்படுத்திகள் தரவுகளை மாற்றவோ அல்லது பக்க விளைவுகளை உருவாக்கவோ முடியும்
   - `ctx.yield_output()` பயன்படுத்தி வேலைப்பாட்டு முடிவுகளை உருவாக்கவும்

### 🚀 உண்மையான உலக பயன்பாடுகள்:

- **பயண முன்பதிவு**: கிடைக்குமா என சரிபார்க்கவும், மாற்று வழிகளை பரிந்துரைக்கவும், விருப்பங்களை ஒப்பிடவும்
- **வாடிக்கையாளர் சேவை**: பிரச்சினை வகை, மனநிலை, முன்னுரிமை அடிப்படையில் வழிமாற்றம் செய்யவும்
- **மின்வணிகம்**: கையிருப்பை சரிபார்க்கவும், மாற்று வழிகளை பரிந்துரைக்கவும், ஆர்டர்களை செயல்படுத்தவும்
- **உள்ளடக்க மிதமாக்கல்**: நச்சுத்தன்மை மதிப்பீடுகள், பயனர் குறியீடுகள் அடிப்படையில் வழிமாற்றம் செய்யவும்
- **அனுமதி வேலைப்பாடுகள்**: தொகை, பயனர் பங்கு, அபாய நிலை அடிப்படையில் வழிமாற்றம் செய்யவும்
- **பல நிலை செயலாக்கம்**: தரவின் தரம், முழுமை அடிப்படையில் வழிமாற்றம் செய்யவும்

### 📚 அடுத்த படிகள்:

- மேலும் சிக்கலான நிபந்தனைகளை சேர்க்கவும் (பல அளவுகோல்கள்)
- வேலைப்பாட்டு நிலை மேலாண்மையுடன் மடக்கங்களை செயல்படுத்தவும்
- மீண்டும் பயன்படுத்தக்கூடிய கூறுகளுக்கான துணை வேலைப்பாடுகளை சேர்க்கவும்
- உண்மையான APIகளுடன் ஒருங்கிணைக்கவும் (ஹோட்டல் முன்பதிவு, கையிருப்பு அமைப்புகள்)
- பிழை கையாளுதல் மற்றும் மாற்று வழிகளை சேர்க்கவும்
- உள்ளமைக்கப்பட்ட காட்சிப்படுத்தல் கருவிகளுடன் வேலைப்பாடுகளை காட்சிப்படுத்தவும்



---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையை பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. எங்கள் தரச்செயல்முறையை உறுதிப்படுத்த முயற்சிக்கிறோம், ஆனால் தானியக்க மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளவும். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
